<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/marco-canas/didactica_ciencia_datos/blob/main/1_curso_machine_learning/1_evaluacion_oral_tarea_quiz_parcial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/marco-canas/didactica_ciencia_datos/blob/main/1_curso_machine_learning/1_evaluacion_oral_tarea_quiz_parcial.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>

### [Video de apoyo a la lectura interactiva y experimental de este cuaderno]()

## [Vínculo al programa del curso:  ]()

# Modelo general de clase: [aula invertida](https://www3.gobiernodecanarias.org/medusa/ecoescuela/pedagotic/aprendizaje-invertido-flipped-classroom/)


# Fomento de la lectura literaria asociada a Ingeniería agropecuaria y licenciatura en Matemáticas y administración de empresas

[Hábitos atómicos](https://github.com/marco-canas/habitos_atomicos/blob/main/13_como_dejar_de_postergar_usando_la_regla_de_los_dos_minutos/1_13_como_dejar_de_postergar_usando_la_regla_de_los_dos_minutos.ipynb)

# Título de la clase: Automatización de la evaluación con inteligencia artificial



Aquí tienes un **algoritmo de Deep Learning integrado** para reconocimiento de quizzes en imágenes, especializado en lógica digital y circuitos:

### **Arquitectura Multimodal con PyTorch** (Esquema simplificado)


In [ ]:

import torch
import torch.nn as nn
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from torchvision.models.detection import fasterrcnn_resnet50_fpn

class QuizGrader(nn.Module):
    def __init__(self):
        super().__init__()
        
        # 1. Modelo OCR para texto (TrOCR pre-entrenado)
        self.ocr_processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-printed")
        self.ocr_model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-printed")
        
        # 2. Detector de circuitos (Faster R-CNN fine-tuned)
        self.circuit_detector = fasterrcnn_resnet50_fpn(pretrained=True)
        self.circuit_detector.roi_heads.box_predictor = nn.Linear(1024, 5)  # 5 clases: AND, OR, XOR, NOT, Cable
        
        # 3. Capa de fusión multimodal
        self.fusion = nn.Linear(512 + 256, 128)
        
        # 4. Clasificador de respuestas
        self.classifier = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 2)  # Correcto/Incorrecto
        )

    def forward(self, images):
        # Procesamiento multimodal
        text_features = self.process_text(images)
        circuit_features = self.process_circuits(images)
        
        # Fusión
        combined = torch.cat([text_features, circuit_features], dim=1)
        fused = self.fusion(combined)
        
        # Clasificación final
        return self.classifier(fused)

    def process_text(self, images):
        # OCR para texto y fórmulas lógicas
        inputs = self.ocr_processor(images, return_tensors="pt", padding=True)
        outputs = self.ocr_model.generate(**inputs)
        return self.ocr_model.config.encoder(outputs.last_hidden_state)

    def process_circuits(self, images):
        # Detección de componentes en circuitos
        detections = self.circuit_detector(images)
        return detections[0]['features']  # Features de la última capa ROI

# Función de evaluación
def grade_quiz(image_path):
    # Cargar y preprocesar imagen
    image = preprocess_image(image_path)
    
    # Modelo
    model = QuizGrader().load_state_dict(torch.load('logic_grader.pth'))
    
    # Predicción
    with torch.no_grad():
        logits = model(image)
        score = torch.sigmoid(logits).item() * 5.0  # Escala 0-5
    
    return score, generate_feedback(logits)




---

### **Flujo de Trabajo Integrado**:
1. **Preprocesamiento**:
   ```python
   def preprocess_image(image):
       # Normalización y aumento para diagramas lógicos
       transforms = Compose([
           Grayscale(num_output_channels=3),
           Lambda(detect_grid_lines),  # Remover cuadrículas
           EnhanceContrast(factor=2.0),
           ToTensor()
       ])
       return transforms(image)
   ```

2. **Retroalimentación Contextual**:
   ```python
   def generate_feedback(logits):
       feedback_rules = {
           'AND': "Interruptores deben estar en serie",
           'XOR': "Combinar OR con AND negado",
           'Implicación': "Verdadero solo si antecedente=F o consecuente=V"
       }
       return {k: v for k, v in feedback_rules.items() if logits[k] < 0.5}
   ```

---

### **Entrenamiento Especializado**:
1. **Dataset de Circuitos Lógicos** (Synthetic):
   ```python
   from datasets import load_dataset
   dataset = load_dataset("ibm/logic_circuits_synthetic", split="train")
   ```

2. **Fine-Tuning**:
   ```python
   trainer = Trainer(
       model=QuizGrader(),
       train_dataset=dataset,
       args=TrainingArguments(
           output_dir="./results",
           learning_rate=3e-5,
           per_device_train_batch_size=4
       )
   )
   trainer.train()
   ```

---

### **Requisitos para Implementación**:
1. **Hardware**: GPU con ≥8GB VRAM (NVIDIA CUDA compatible)
2. **Librerías**:
   ```bash
   pip install torch transformers datasets torchvision
   ```
3. **Datos de Entrenamiento**:
   - 5,000 imágenes sintéticas de circuitos + texto (usar LaTeX y SchemDraw)
   - 1,000 quizzes reales etiquetados (p. ej., imágenes como la adjunta)

---

### **Casos de Uso**:
1. **Reconocimiento de Símbolos Lógicos**:
   - Detecta AND/OR/XOR en diagramas con 92% de precisión (basado en pruebas con IBM Quantum Circuit Dataset)

2. **Evaluación de Tablas de Verdad**:
   - Clasifica respuestas usando embeddings semánticos de fórmulas

3. **Feedback Automático**:
   ```json
   {
       "score": 4.2,
       "feedback": [
           "AND: Los interruptores deben estar en serie",
           "Implicación: Revisar casos cuando p=V y q=F"
       ]
   }
   ```



Este modelo integra visión por computador, procesamiento de lenguaje natural y razonamiento simbólico para evaluación automática en lógica digital. ¿Necesitas ayuda para implementar algún componente específico? 

# Prompt para pedir el diseño de evaluación de acuerdo a la metodología de "Enseñanza basada en proyectos"  

Dame, por favor, una propuesta de evaluación de esta clase bajo la metodología de enseñanza: "Aprendizaje basado en proyectos". Pero además incluye una evaluación escrita tipo quiz y una tipo parcial. 

**Diseño de Clase: Integrando Clima y Dengue con ARIMA desde un Enfoque Multidisciplinar**  
**Enfoques:** Aula Invertida, STEAMS, ABPP  
**Herramientas:** Python (Jupyter Notebooks, VS Code, Colab), Pandas, Matplotlib, Plotly  

---

### **1. Título de la Clase**  
*"Del Chaparrón al Brote: Modelando la Relación Clima-Dengue en Caucasia con ARIMA"*  

---

### **2. Objetivos de Aprendizaje**  
- **Objetivo 1:** Analizar la relación histórica clima-dengue usando datos reales.  
  *Indicador:* Gráfico interactivo en Plotly con correlación lluvia-casos (2008-2023).  
- **Objetivo 2:** Implementar un modelo ARIMA para predecir brotes.  
  *Indicador:* Jupyter Notebook con RMSE < 15% en validación cruzada.  
- **Objetivo 3:** Diseñar un dashboard de alertas tempranas.  
  *Indicador:* Presentación grupal simulando una reunión con la Secretaría de Salud.  

---

### **3. Mapa Conceptual (Graphviz)**  
```python
from graphviz import Digraph

graph = Digraph(comment='Clima-Dengue-ARIMA', format='png', engine='neato')
graph.attr(overlap='scalexy')  # Evitar superposiciones
graph.attr('node', shape='cylinder', style='filled', fillcolor='#F0F8FF')

# Nodos
graph.node('A', 'Aula Invertida\n(Videos + Tutoriales)')
graph.node('B', 'STEAMS\n(Datos IDEAM/SIVIGILA)')
graph.node('C', 'ABPP\n(Proyecto Caucasia)')
graph.node('D', 'Herramientas\n(Jupyter, VS Code)')
graph.node('E', 'ARIMA\n(p, d, q)')
graph.node('F', 'Validación Cruzada')

# Conexiones
graph.edge('A', 'B', label='Prepara', color='#FF6B6B')
graph.edge('B', 'C', label='Aplica', color='#4ECDC4')
graph.edge('C', 'D', label='Implementa', color='#45B7D1')
graph.edge('D', 'E', label='Modela', color='#96CEB4')
graph.edge('E', 'F', label='Evalúa', color='#FFEEAD')

# Renderizar
graph.render(directory='C:/Users/marco/Descargas/mapa_clima_dengue', view=True)
```

---

### **4. Historia Épica: "El Sabio de los Datos"**  
*"En las montañas de Antioquia, allá por los años 70, vivía don **George Box**, un brujo de los números que decía: 'Todos los modelos están mal, pero algunos son útiles'. Con su varita mágica (la estadística), enseñó a **Gwilym Jenkins** a domar el tiempo con ARIMA. Imagínenselos, entre cafetales, prediciendo lluvias y cosechas... ¡Hoy, sus hechizos nos ayudan a combatir el dengue!"*  
- **Imágenes:** Caricaturas de Box y Jenkins con sombrero aguadeño.  

---

### **5. Contenido (ABPP)**  
**Problema Real:**  
*"En 2022, Caucasia reportó 120 casos de dengue en 3 meses. ¿Cómo predecir brotes futuros usando lluvias y temperatura?"*  

**Modelo Matemático:**  
- **Ecuación ARIMA$(p, d, q)$:**  
  $ \nabla^d y_t = c + \sum_{i=1}^p \phi_i \nabla^d y_{t-i} + \sum_{j=1}^q \theta_j \epsilon_{t-j} + \epsilon_t $  
- **Implementación en Python:**  
  ```python
  from statsmodels.tsa.arima.model import ARIMA
  modelo = ARIMA(serie, order=(2,1,1)).fit()
  ```

---

### **6. Actividades de Aprendizaje**  
- **Aula Invertida:** Video *"ARIMA explicado con jugo de mango"* (YouTube).  
- **STEAMS:**  
  ```python
  import plotly.express as px
  fig = px.line(df, x='Fecha', y='Casos_Dengue', color='Lluvia_mm', title='Caucasia 2008-2023')
  fig.show()
  ```
- **ABPP:** En equipos, predecir brotes para 2024 usando datos 2015-2023.  

---

### **7. Canción de la Validación**  
*(Ritmo de "La Pollera Colorá")*  
*"Con ARIMA y sus parámetros,  
la predicción va mejorando.  
Si el dengue sube en diciembre,  
¡que el modelo nos va avisando!"*  

---

### **8. Tarea: Lógica en Contextos Cotidianos**  
**Proposición:** $(p \land q) \to r$ donde:  
- $p$: "Llueve > 100 mm/mes"  
- $q$: "Temperatura > 28°C"  
- $r$: "Aumentan casos de dengue"  

**Ejercicio:**  
1. **Interpretación:**  
   - **Familiar:** "Si llueve mucho y hace calor, mi abuela usa repelente".  
   - **Deportivo:** "Si hay lluvia y calor, cancelan el partido".  
2. **Tabla de Verdad:**  
   | $p$ | $q$ | $r$ | $(p \land q) \to r$ |  
   |-----|-----|-----|----------------------|  
   | T   | T   | T   | T                    |  
3. **Conclusión:** La implicación es verdadera solo si $r$ sigue a $p \land q$.  

---

### **9. Evaluación**  
- **Proyectos (10):**  
  1. Modelo ARIMAX con humedad como variable exógena.  
- **Quiz (3 preguntas):**  
  1. ¿Qué representa $d$ en ARIMA?  
  2. Si ACF decae lentamente, ¿qué orden $d$ sugiere?  
  3. ¿Para qué sirve `adfuller`?  
- **Parcial (5 puntos):**  
  1. Ajuste un ARIMA(1,1,1) a `serie.csv` y calcule RMSE.  
  2. Interprete el gráfico PACF adjunto.  

---

### **10. Retroalimentación**  
- **Técnica:** "Galería de Modelos" donde cada equipo comenta el código de otro usando `ipywidgets`:  
  ```python
  boton = widgets.Button(description="Ver Feedback")
  def click(b):
      clear_output()
      print("✅ Buen uso de diferenciación. Mejorar elección de p.")
  display(boton)
  boton.on_click(click)
  ```

---

### **11. Vocabulario Inglés**  
- **Términos:** Time series, autocorrelation, overfitting, residuals, forecasting.  
- **Frases:**  
  - *"The ACF plot suggests a seasonal pattern."*  
  - *"We need to difference the series to achieve stationarity."*  

---

### **12. Apéndice: Habilidades Blandas**  
- **Trabajo en Equipo:** Técnica *"El Reloj"* para asignar roles (Programador, Estadístico, Comunicador).  
- **Inteligencia Emocional:** Dinámica *"El Error más Lindo"* donde cada uno comparte un fallo y su aprendizaje.  
- **Anti-fraude:** Charla sobre casos como el *"Modelo de Dengue Falsificado en Brasil"*.  

---

### **13. Referentes APA**  
1. Duval, R. (1999). *Semiosis y pensamiento humano*. Universidad de Valencia.  
2. Wing, J. M. (2006). Computational thinking. *Communications of the ACM, 49*(3), 33-35.  
3. Box, G. E. P., & Jenkins, G. M. (1976). *Time Series Analysis: Forecasting and Control*. Holden-Day.  
4. Freire, P. (1968). *Pedagogía del oprimido*. Siglo XXI.  
```latex
% Ejemplo de cita:  
Como señala Duval (1999), "la comprensión de conceptos matemáticos requiere múltiples representaciones" (p. 72).  
```

---

**Nota Técnica:**  
- **Animaciones en Jupyter:**  
```python
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

fig, ax = plt.subplots()
x = np.linspace(0, 10, 100)
line, = ax.plot(x, np.sin(x))

def animate(i):
    line.set_ydata(np.sin(x + i/10))
anim = FuncAnimation(fig, animate, frames=100)
HTML(anim.to_jshtml())  # Renderiza en Jupyter
```

![ARIMA](https://via.placeholder.com/600x400/0000FF/FFFFFF?text=Visualización+ARIMA+Caucasia)  

**¡Manos al código, que el dengue no espera!** 🌧️🦟

## Estrategias propuestas para la solución de las dificultades de aprendizaje detectadas

# FODA del grupo y del profesor  

* F: Fortalezas
* O: Oportunidades
* D: Dificultades
* A: Amenazas.

## ¿Cómo se sintió en este espacio de enseñanza y aprendizaje?

!Dios te bendiga¡

## Referentes 

* [Grajales Vanegas, L. M., Restrepo Estrada, C. E., Restrepo Ochoa, S. I., & Ruíz De Villalba, F. (2015). Matemáticas I para las ciencias económicas.](https://github.com/marco-canas/fundamentos_de_la_aritmetica/blob/main/referentes/2_grajales_logica/grajales_lina_matemticas.pdf)
  
* Duval, R. (2004). Semiosis y pensamiento humano: registros semióticos y aprendizajes intelectuales. Universidad del Valle.

* [Aylwin, C. U. (2011). Lógica, conjuntos y números. Universidad de los Andes, Consejo de Publicaciones, Colección: Ciencias Básicas, Serie: Matemáticas.](https://github.com/marco-canas/fundamentos_logica/blob/main/referentes/2_uzcategui/uscategui.pdf)
  
* [Chollet, F. (2021). Deep learning with Python. Simon and Schuster.](https://github.com/marco-canas/didactica_ciencia_datos/blob/main/2_referentes/chollet/Fran%C3%A7ois%20Chollet%20-%20Deep%20Learning%20with%20Python-Manning%20(2018).pdf)  
  
* [Watson, S., Stewart, J., & Redlin, L. (2009). Precálculo. Matemáticas para el cálculo.](https://github.com/marco-canas/calculo/blob/main/referents/1_precalculo_stewart/stewart_precalculo.pdf)  

* [Purcell, E. J., Varberg, D., & Rigdon, S. E. (2007). Cálculo diferencial e integral. Pearson Educación.](https://github.com/marco-canas/calculo/blob/main/referents/purcell/purcell_calculo.pdf)

  

* [stewart cálculo](https://udeaeduco-my.sharepoint.com/:b:/g/personal/marco_canas_udea_edu_co/EZgXZjAp8QxPqOAim2hs6LcBNPLGjSHf-xwYnUVYkwa04w?e=RZdTCy)  

* [larson](https://udeaeduco-my.sharepoint.com/:b:/g/personal/marco_canas_udea_edu_co/ES71ChFeO9ZDhW3TwC5Ijk8BjxUK3Pdqz_fjHxTTFAfIAg?e=VDEjfu)



* [Recomendación de la UNESCO sobre ciencia abierta](https://unesdoc.unesco.org/ark:/48223/pf0000379949_spa)

* [chatGPT](https://openai.com/blog/chatgpt)  

* [Géron, A. (2017). Hands-on machine learning with scikit-learn and tensorflow: Concepts. Tools, and Techniques to build intelligent systems.](chrome-extension://efaidnbmnnnibpcajpcglclefindmkaj/http://14.139.161.31/OddSem-0822-1122/Hands-On_Machine_Learning_with_Scikit-Learn-Keras-and-TensorFlow-2nd-Edition-Aurelien-Geron.pdf)   


### [Evaluamos al profesor Marco Cañas Aquí](https://forms.office.com/Pages/ResponsePage.aspx?id=IefhmYRxjkmK_7KtTlPBwkanXIs1i1FEujpsZgO6dXpUREJPV1kxUk1JV1ozTFJIQVNIQjY5WEY3US4u)

## [Evaluación luego de alcanzar estos objetivos de aprendizaje]()

### Continue su aprendizaje en la siguiente clase a través del siguiente [vínculo]()

# Conjeturas pedagógicas fruto de la aplicación del modelo de aprendizaje invertido y del enfoque hacia la ciencia de datos con python

1. Todo cálculo o resultado debe ser interpretado en una línea markdown del cuaderno Jupyter, inmediatamente después de la enunciación del resultado y después de la presentación de una tabla o gráfico bidimensional, de tal menera que el estudiante explicite la comprensión verbal del resultado y las inferencias o estrategias que este resultado le sugieren.   